#### This Code fetches protein GPCR family directly from PDB and extracts sequence, dssp3, dssp8 structure and saved as txt format and text to csv saved format#######

Extraction of Dataset Protein GPCR family from PDB and conversion of DSSP8 to DSSP3 saving as csv file
This Code fetches protein GPCR family directly from PDB

In [1]:
from IPython import get_ipython
from IPython.display import display
# # %%
!pip install biopython
# # %%
# # Install DSSP (mkdssp)
!sudo apt-get update
!sudo apt-get install dssp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 22.7 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,757 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,103 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 k

You can input any family classified PDB protein family in place of "Kinase"

In [2]:
import requests

def search_pdb_gpcr_structures(n=50):
    url = "https://search.rcsb.org/rcsbsearch/v2/query"

    # Query for PDB entries classified as GPCRs
    query = {
        "query": {
            "type": "terminal",
            "service": "text",
            "parameters": {
                "attribute": "struct_keywords.pdbx_keywords",
                "operator": "contains_words",
                "value": "G protein-coupled receptor"
            }
        },
        "return_type": "entry",
        "request_options": {
            "results_content_type": ["experimental"],
            "sort": [
                {
                    "sort_by": "score",
                    "direction": "desc"
                }
            ],
            "paginate": {
                "start": 0,
                "rows": n
            }
        }
    }

    response = requests.post(url, json=query)

    if response.status_code != 200:
        print("Error:", response.status_code, response.text)
        return []

    data = response.json()

    if 'result_set' not in data:
        print("No results found.")
        return []

    pdb_ids = [item['identifier'] for item in data['result_set']]
    return pdb_ids

# Test it
gpcr_pdb_ids = search_pdb_gpcr_structures(50)
print("GPCR PDB IDs:", gpcr_pdb_ids)


GPCR PDB IDs: ['1AJE', '1E0S', '1RGP', '1ZBD', '2CJW', '2RAP', '5NI8', '5NIB', '1H30', '1A52', '1AC6', '1AJJ', '1BEC', '1BJ8', '1E6F', '1GP0', '1GP3', '1GQB', '1H68', '1HKF', '1M6P', '1NGR', '1RPM', '1TCR', '1UPV', '1UPW', '1US4', '1US5', '1W9R', '2BPD', '2BPE', '2BPH', '2BSD', '2BSE', '2BYN', '2BYP', '2BYQ', '2BYR', '2BYS', '2CDG', '2CL8', '2CNJ', '2J4A', '2J67', '2NR1', '2RQZ', '2RR0', '2RR2', '2UV3', '2UWI']


In [3]:
from Bio.PDB import PDBList, PDBParser, DSSP

# === Map 8-state DSSP to Q3 ===
def map_dssp8_to_q3(dssp8_char):
    if dssp8_char in ['H', 'G', 'I']:
        return 'H'
    elif dssp8_char in ['E', 'B']:
        return 'E'
    else:
        return 'C'

# === DSSP Extraction Function ===
def get_dssp_info(pdb_id, dssp_exe='mkdssp'):
    pdb_id = pdb_id.lower()
    pdbl = PDBList()
    pdbl.retrieve_pdb_file(pdb_id, pdir='.', file_format='pdb')
    pdb_file = f"pdb{pdb_id}.ent"

    parser = PDBParser()
    structure = parser.get_structure(pdb_id, pdb_file)
    model = structure[0]

    dssp = DSSP(model, pdb_file, file_type='PDB', dssp=dssp_exe)

    dssp_keys = list(dssp.keys())
    primary_sequence = ''.join([dssp[key][1] for key in dssp_keys])
    dssp8_string = ''.join([dssp[key][2] for key in dssp_keys])
    dssp3_string = ''.join([map_dssp8_to_q3(dssp[key][2]) for key in dssp_keys])

    return primary_sequence, dssp8_string, dssp3_string

# === GPCR PDB IDs ===
gpcr_pdb_ids = [
    '1AJE', '1E0S', '1RGP', '1ZBD', '2CJW', '2RAP', '5NI8', '5NIB', '1H30', '1A52',
    '1AC6', '1AJJ', '1BEC', '1BJ8', '1E6F', '1GP0', '1GP3', '1GQB', '1H68', '1HKF',
    '1M6P', '1NGR', '1RPM', '1TCR', '1UPV', '1UPW', '1US4', '1US5', '1W9R', '2BPD',
    '2BPE', '2BPH', '2BSD', '2BSE', '2BYN', '2BYP', '2BYQ', '2BYR', '2BYS', '2CDG',
    '2CL8', '2CNJ', '2J4A', '2J67', '2NR1', '2RQZ', '2RR0', '2RR2', '2UV3', '2UWI'
]

# === LOOP OVER PDB IDs AND SAVE RESULTS ===
output_lines = []

for pdb_id in gpcr_pdb_ids:
    print(f"Processing {pdb_id}...")
    try:
        primary_seq, dssp8_str, dssp3_str = get_dssp_info(pdb_id)

        print(f"  → Lengths: Primary={len(primary_seq)}, Q8={len(dssp8_str)}, Q3={len(dssp3_str)}")

        entry = (
            f"PDB ID: {pdb_id}\n"
            f"Primary sequence:\n{primary_seq}\n"
            f"DSSP 8-state (Q8):\n{dssp8_str}\n"
            f"DSSP 3-state (Q3):\n{dssp3_str}\n"
            f"Lengths: Primary={len(primary_seq)}, Q8={len(dssp8_str)}, Q3={len(dssp3_str)}\n"
            + "-"*60 + "\n"
        )
        output_lines.append(entry)

    except Exception as e:
        print(f"  ✖ Error processing {pdb_id}: {e}")
        output_lines.append(f"PDB ID: {pdb_id} → ERROR: {e}\n{'-'*60}\n")

# === WRITE TO FILE ===
with open("gpcr_dssp_results.txt", "w") as f:
    f.writelines(output_lines)

print("\n✅ All results saved to 'gpcr_dssp_results.txt'")


Processing 1AJE...
  → Lengths: Primary=194, Q8=194, Q3=194
Processing 1E0S...
  → Lengths: Primary=173, Q8=173, Q3=173
Processing 1RGP...
  → Lengths: Primary=189, Q8=189, Q3=189
Processing 1ZBD...


/usr/local/lib/python3.11/dist-packages/Bio/PDB/StructureBuilder.py:100: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 3059.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/Bio/PDB/StructureBuilder.py:100: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 3092.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/Bio/PDB/StructureBuilder.py:100: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 3094.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/Bio/PDB/StructureBuilder.py:100: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 3125.
  warnings.warn(


  → Lengths: Primary=301, Q8=301, Q3=301
Processing 2CJW...


/usr/local/lib/python3.11/dist-packages/Bio/PDB/StructureBuilder.py:100: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 3485.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/Bio/PDB/StructureBuilder.py:100: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 3514.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/Bio/PDB/StructureBuilder.py:100: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 3543.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/Bio/PDB/StructureBuilder.py:100: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 3624.
  warnings.warn(


  → Lengths: Primary=351, Q8=351, Q3=351
Processing 2RAP...
  → Lengths: Primary=167, Q8=167, Q3=167
Processing 5NI8...


...

  → Lengths: Primary=275, Q8=275, Q3=275
Processing 2NR1...
  → Lengths: Primary=23, Q8=23, Q3=23
Processing 2RQZ...
  → Lengths: Primary=38, Q8=38, Q3=38
Processing 2RR0...
  → Lengths: Primary=38, Q8=38, Q3=38
Processing 2RR2...
  → Lengths: Primary=38, Q8=38, Q3=38
Processing 2UV3...


/usr/local/lib/python3.11/dist-packages/Bio/PDB/StructureBuilder.py:100: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 2452.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/Bio/PDB/StructureBuilder.py:100: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 2489.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/Bio/PDB/StructureBuilder.py:100: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 2506.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/Bio/PDB/StructureBuilder.py:100: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 2658.
  warnings.warn(


  → Lengths: Primary=242, Q8=242, Q3=242
Processing 2UWI...


/usr/local/lib/python3.11/dist-packages/Bio/PDB/StructureBuilder.py:100: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 2329.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/Bio/PDB/StructureBuilder.py:100: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 2415.
  warnings.warn(


  → Lengths: Primary=254, Q8=254, Q3=254

✅ All results saved to 'gpcr_dssp_results.txt'


In [4]:
import csv

input_file = "gpcr_dssp_results.txt"
output_file = "gpcr_dssp_results.csv"

entries = []

with open(input_file, "r") as f:
    lines = f.read().split("------------------------------------------------------------\n")

    for block in lines:
        if not block.strip():
            continue

        lines = block.strip().splitlines()

        try:
            pdb_id = lines[0].split(":")[1].strip()
            primary_seq = lines[2]
            dssp_q8 = lines[4]
            dssp_q3 = lines[6]
            length_line = lines[7]
            length = length_line.split("Primary=")[1].split(",")[0].strip()

            entries.append([pdb_id, primary_seq, dssp_q8, dssp_q3, length])
        except Exception as e:
            print(f"Skipping a block due to parsing error: {e}")

# Write to CSV
with open(output_file, "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["PDB_ID", "Primary_Sequence", "DSSP_Q8", "DSSP_Q3", "Length"])
    writer.writerows(entries)

print(f"✅ CSV file saved as '{output_file}'")


✅ CSV file saved as 'gpcr_dssp_results.csv'
